In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# The Task
In this notebook, we are trying to create an algorithm to classify a wide range (7, to be precise) of emotions based on facial expressions. For image classification, we have used the AlexNet DCNN.


# Why AlexNet?
AlexNet was the winner of the 2012 ImageNet challenge. AlexNet had a remarkable intervention of using **relu** activation function to increase the efficiency by over 6 times by reducing the chances of Vanishing Gradient (VG) problems. 
Another advantage AlexNet has is that overlappping maxpooling layers considerably improve model top-1 and top-5 accuracies.

The model consists of a total of 8 layers: five layers with a combination of max pooling followed by 3 fully connected layers. AlexNet was revolutionary in its field because it was the first model of its kind to introduce consecutive convolution layers in its architecture. 
![AlexNet Architecture](https://www.researchgate.net/profile/Nicola-Strisciuglio/publication/339756908/figure/fig5/AS:866265283457032@1583545146587/AlexNet-architecture-used-as-the-baseline-model-for-the-analysis-of-results-on-the.png)

It is important to note that AlexNet accepts input images of size: **227X227X3**






# Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import numpy as np
import pathlib
import datetime

# printout versions
print(f"Tensor Flow Version: {tf.__version__}")
print(f"numpy Version: {np.version.version}")

# Generating list of train images, classes and Class Names

In [ ]:
data_dir = pathlib.Path("/kaggle/input/face-emotion-dataset/archive/train")
image_count = len(list(data_dir.glob('*/*.png')))
print(image_count)
# classnames in the dataset specified
CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt" ])
print(CLASS_NAMES)
# print length of class names
output_class_units = len(CLASS_NAMES)
print(output_class_units)


In [ ]:
print(CLASS_NAMES)

# Creating the AlexNet model

In [ ]:
model = tf.keras.models.Sequential([
    # 1st conv
  tf.keras.layers.Conv2D(96, (11,11),strides=(4,4), activation='relu', input_shape=(227, 227, 3)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2,2)),
    # 2nd conv
  tf.keras.layers.Conv2D(256, (11,11),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
     # 3rd conv
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 4th conv
  tf.keras.layers.Conv2D(384, (3,3),strides=(1,1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
    # 5th Conv
  tf.keras.layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu',padding="same"),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2, strides=(2, 2)),
  # To Flatten layer
  tf.keras.layers.Flatten(),
  # To FC layer 1
  tf.keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  #To FC layer 2
  tf.keras.layers.Dense(4096, activation='relu'),
    # add dropout 0.5 ==> tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(output_class_units, activation='sigmoid')
])

# Creating Data Generator Objects

In [ ]:
data_dir2 = pathlib.Path("/kaggle/input/face-emotion-dataset/archive/test")

BATCH_SIZE = 32             # Can be of size 2^n, but not restricted to. for the better utilization of memory
IMG_HEIGHT = 227            # input Shape required by the model
IMG_WIDTH = 227             # input Shape required by the model
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

# Rescalingthe pixel values from 0~255 to 0~1 For RGB Channels of the image.
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# training_data for model training
train_data_gen = image_generator.flow_from_directory(directory=str(data_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH), #Resizing the raw dataset
                                                     classes = list(CLASS_NAMES))

val_data_gen = image_generator.flow_from_directory(directory=str(data_dir2),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH), #Resizing the raw dataset
                                                     classes = list(CLASS_NAMES))

In [ ]:
!pip install tensorflow-addons

# Compiling and Summarizing the Model

In [ ]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.SensitivityAtSpecificity(0.5), tf.keras.metrics.SpecificityAtSensitivity(0.5), tf.keras.metrics.AUC(curve='ROC')])

# Summarizing the model architecture and printing it out
model.summary()

# Training the Model on 10 Epochs

In [ ]:
import time
start = time.time()
history = model.fit(
      train_data_gen,
      steps_per_epoch=STEPS_PER_EPOCH,
      epochs=10,
    validation_data=val_data_gen
    
)

# Saving the model
model.save('AlexNet_saved_model/')
print("Total time: ", time.time() - start, "seconds")

# Plotting the Metrics

In [ ]:
def plot_hist(history):
#     plt.plot(history.history["val_sensitivity_at_specificity"], color='red')
#     plt.plot(history.history["val_auc"], color='blue')
#     plt.plot(history.history["val_specificity_at_sensitivity"], color='orange')
#     plt.plot(history.history["val_accuracy"],color='green')
#     plt.plot(history.history["accuracy"],color='olive')
#     plt.plot(history.history["precision"],color='violet')
#     plt.plot(history.history["val_precision"],color='purple')
#     plt.plot(history.history["recall"],color='cyan')
#     plt.plot(history.history["val_recall"],color='yellow')


    
    

     
    
    
    plt.title("Metrics")
    
    
    plt.legend(["Sensitivity","AUC","Specificity", "Test Accuracy", "Train Accuracy", "Precision", "Test Precision", "Recall", "Test Recall" ], bbox_to_anchor =(0.65, 1.00))
    plt.show()


plot_hist(history)


# Thank you! Do check out my other kernels on Skin Cancer!

In [ ]:
!ls /kaggle/working/AlexNet_saved_model

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('AlexNet_saved_model/')

# Now you can use the loaded model to make predictions on new data
# For example, if you have a test generator named test_data_gen
# (make sure it's set up similarly to your training and validation generators),
# you can use the following code:

predictions = loaded_model.predict(val_data_gen)

# 'predictions' will contain the model


In [ ]:
evaluation = loaded_model.evaluate(val_data_gen)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])


In [ ]:
import shutil

# Replace 'your_directory' with the path to the directory you want to compress
directory_to_compress = '/kaggle/working/AlexNet_saved_model'

# Replace 'compressed_file.zip' with the desired name of the compressed file
compressed_file_path = '/kaggle/working/compressed_file.zip'

# Create a zip file from the directory
shutil.make_archive(compressed_file_path, 'zip', directory_to_compress)
